# Määruste tabel


Tabel on tehtud Heiki-Jaan Kaalepi poolt
Pärit siit:

https://kodu.ut.ee/~hkaalep/kertule/

## 1. Lahti pakkimine, esialgne filtreerimine awk'iga

Kokku on 11 tulpa. Tulbad on kirjeldatud [siin](./source/v11_README.pdf).

In [1]:
!pwd

/Users/rabauti/repos/eki/prg1978/advtypes


In [2]:
# lahtipakkimine, originaal säilib
!gzip -dk ./source/v11_tabel_m3.csv.gz ./

gzip: ./ is a directory


In [3]:
# ridade arv
!wc -l ./source/v11_tabel_m3.csv

 4649813 ./source/v11_tabel_m3.csv


In [4]:
# sisu vaatamine
!cat ./source/v11_tabel_m3.csv | head -10 

1;olema;;;;;;;;;;2308619
2;toimuma;;lõpp;in;;;;;match;ajam;331
3;toimuma;;1.;<käändumatu>;;;;;;;103
4;saama;pihta;keel;all;;;;;;;1
5;kulmineeruma;;purukspeksmine;kom;;;;;;;1
6;lõpetama;;;;;;;;;;6875
7;tulema;;sina;ad;;;;;;;1144
8;viilima;;tund;el;;;;;;ajam;3
9;viilima;;juht;ad;;;;;;;1
10;tulema;;mis;gen;kõrvalt;;;;;koham;2
cat: stdout: Broken pipe


Eraldajaks on ;
Kohamääruse info asub 11-s veerus.

In [5]:
# võimalikud kohamääruse väärtused koos sagedusega
!awk -F ';' '{print $11}' ./source/v11_tabel_m3.csv | sort -d | uniq -c

2203508 
601981 ajam
1182011 koham
32368 omajam
30244 puuviga
599701 tundmatum


In [6]:
# sisse ainult need read, kus kohamäärusel on väärtusteks ajam, koham või omajam
!awk -F ';' '{if (($11=="ajam") || ($11=="koham")  || ($11=="omajam") ) {print}}' ./source/v11_tabel_m3.csv > ./source/v11_tabel_m3_filtered.csv

In [7]:
# kotroll, et sagedused jäid samaks
!awk -F ';' '{print $11}' ./source/v11_tabel_m3_filtered.csv | sort -d | uniq -c

601981 ajam
1182011 koham
32368 omajam


## 2. Tabeli töötlemine Python pandas teegiga

Tulpade nimetuste info võetud [siit](./source/v11_README.pdf).


In [8]:
import pandas as pd

names = (
    'id',
    'verb', 
    'verb_compound', 
    'obl_root', 
    'obl_case',
    'obl_k',
    'ner_loc',
    'ner_per',
    'ner_org',
    'timex',
    'advtype',
    'count',
)
df = pd.read_csv('./source/v11_tabel_m3_filtered.csv', sep=';', header=None, names=names, index_col="id")
df.fillna("",inplace=True)
df.shape

(1816360, 11)

In [9]:
df.columns

Index(['verb', 'verb_compound', 'obl_root', 'obl_case', 'obl_k', 'ner_loc',
       'ner_per', 'ner_org', 'timex', 'advtype', 'count'],
      dtype='object')

In [10]:
df.head(10)
#obl_type_df.sort_values('count', ascending=False).head(10)

,verb,verb_compound,obl_root,obl_case,obl_k,ner_loc,ner_per,ner_org,timex,advtype,count
id,,,,,,,,,,,
2,toimuma,,lõpp,in,,,,,match,ajam,331
8,viilima,,tund,el,,,,,,ajam,3
10,tulema,,mis,gen,kõrvalt,,,,,koham,2
18,jõudma,,aasta,gen,algul,,,,match,ajam,35
27,kutsuma,,elu,adit,,,,,,koham,113
28,tulema,,toim,adit,,,,,,koham,2508
39,loobuma,,aeg,ad,,,,,match,ajam,19
49,jooma,,hommik,el,,,,,match,ajam,3
50,jooma,,õhtu,term,,,,,,ajam,4


In [11]:
df.fillna("",inplace=True)
df[ 
    (df['verb'] == 'seisnema') & 
    (df['verb_compound'] == '') &
    (df['obl_case'] == 'in') &
    (df['obl_root'] == 'see') &
    (df['obl_k'] == '')
    ].sort_values('count', ascending=False).head(10)

,verb,verb_compound,obl_root,obl_case,obl_k,ner_loc,ner_per,ner_org,timex,advtype,count
id,,,,,,,,,,,
2869,seisnema,,see,in,,,,,,koham,2775
30890,seisnema,,see,in,,contains,,,,koham,237
18768,seisnema,,see,in,,,,,contains,koham,214
56551,seisnema,,see,in,,,,contains,,koham,168
53855,seisnema,,see,in,,,contains,,,koham,86
209317,seisnema,,see,in,,contains,,,contains,koham,43
188275,seisnema,,see,in,,,,contains,contains,koham,34
77714,seisnema,,see,in,,contains,,contains,,koham,30
43750,seisnema,,see,in,,contains,contains,,,koham,24


In [12]:
# uus andmestik vajalikke veergudega
df2 = df[['verb', 'verb_compound', 'obl_root', 'obl_case', 'obl_k', 'advtype', 'count']]

df2[ 
    (df2['verb'] == 'seisnema') & 
    (df2['verb_compound'] == '') &
    (df2['obl_case'] == 'in') &
    (df2['obl_root'] == 'see') &
    (df2['obl_k'] == '')
    ].sort_values('count', ascending=False).head(10)

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
id,,,,,,,
2869,seisnema,,see,in,,koham,2775
30890,seisnema,,see,in,,koham,237
18768,seisnema,,see,in,,koham,214
56551,seisnema,,see,in,,koham,168
53855,seisnema,,see,in,,koham,86
209317,seisnema,,see,in,,koham,43
188275,seisnema,,see,in,,koham,34
77714,seisnema,,see,in,,koham,30
43750,seisnema,,see,in,,koham,24


In [13]:
# grupeerime read 'verb', 'verb_compound', 'obl_root', 'obl_case', 'obl_k', 'advtype' järgi
# summeerime count
df_grouped = df2.groupby(['verb', 'verb_compound', 'obl_root', 'obl_case', 'obl_k', 'advtype']).agg({'count': 'sum'}).reset_index()
df_grouped.shape

(1558489, 7)

In [14]:
df_grouped.head()

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
0,0olema,,kuu,term,,ajam,1
1,0olema,,pood,in,,koham,1
2,10halama,,faks,in,,koham,1
3,1811vastama,,keel,in,,koham,1
4,1811vastama,,kell,part,,ajam,1


In [15]:
df_grouped[ 
    (df_grouped['verb'] == 'seisnema') & 
    (df_grouped['verb_compound'] == '') &
    (df_grouped['obl_case'] == 'in') &
    (df_grouped['obl_root'] == 'see') &
    (df_grouped['obl_k'] == '')
    ].sort_values('count', ascending=False).head(10)

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
1079691,seisnema,,see,in,,koham,3668


In [16]:
# tulemuse salvestamine
df_grouped.to_csv('./result/advtype_df.csv', index=False, sep='\t')


## c3. Kasulikud käsud

In [17]:

df_grouped[ 
    (df_grouped['verb'] == 'väljenduma') & 
    (df_grouped['verb_compound'] == '') &
    (df_grouped['obl_case'] == 'in') &
    (df_grouped['obl_root'] == 'arhitektuur') &
    (df_grouped['obl_k'] == '')
    ]

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
1491257,väljenduma,,arhitektuur,in,,koham,5


In [18]:
df_grouped.fillna("",inplace=True)
df_grouped[ 
    (df_grouped['verb'] == 'tulema') & 
    (df_grouped['verb_compound'] == '') &
    (df_grouped['obl_case'] == 'ad') &
    (df_grouped['obl_root'] == 'korraldamine') &
    (df_grouped['obl_k'] == '')
    ]

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
1308325,tulema,,korraldamine,ad,,ajam,22
1308326,tulema,,korraldamine,ad,,koham,1


In [19]:
df_grouped.fillna("",inplace=True)
df_grouped[ 
    (df_grouped['verb'] == 'seisnema') & 
    (df_grouped['verb_compound'] == '') &
    (df_grouped['obl_case'] == 'in') &
    (df_grouped['obl_root'] == 'see') &
    (df_grouped['obl_k'] == '')
    ]

,verb,verb_compound,obl_root,obl_case,obl_k,advtype,count
1079691,seisnema,,see,in,,koham,3668
